In [38]:
#사용할 패키지를 import한다.
#크롤링에는 selenium과 beautifulsoup4를 사용했다.
from urllib.request import urlopen
from bs4 import BeautifulSoup
from selenium import webdriver as wd
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from fake_useragent import UserAgent
import openpyxl
import time
import pandas as pd
import numpy as np

# fake_useragent의 UserAgent 메서드를 통해 랜덤한 UserAgent로 서버가 인식되게 한다.
# 서버가 크롤러를 봇으로 인식하여 차단하는 것을 방지하기 위해서다.
options = Options()
ua = UserAgent()
userAgent = ua.random
print(userAgent)
options.add_argument(f'user-agent={userAgent}')
driver = wd.Chrome(chrome_options=options)

# url에 2022년도 무신사 리뷰 페이지를 담아준다.
url = "https://www.musinsa.com/app/reviews/lists?type=&year_date=2022&month_date=&day_date=&max_rt=2023&min_rt=2009&brand=&page=1&sort=&hash_id=&best_type=&s_type=all&q="
# selenium library를 통해 웹페이지를 열어준다.
driver.get(url)

time.sleep(10)

# 20000개의 무신사 리뷰 데이터를 담을 dataframe을 만들어줬다.
# 'profile_body_information' : 체형 정보 (성별, 키, 몸무게)
# 'product_name' : 제품명
# 'brand_name' : 브랜드명
# 'review_text' : 리뷰 본문
# 'review_rating' : 별점 'width가 100%면 별 5개, 20%당 1개'

review_df = pd.DataFrame('', np.array(range(1,20001)), 
                        columns= ['profile_body_information', 'product_name','brand_name', 'review_text', 'review_rating', ''])

# 리뷰 아이템 수를 n에 저장한다.
n = 0
# 상품 image들의 url을 담을 리스트를 만들었다.
img_Url = []

# 1페이지당 20개여서 1000개 페이지에 대해 반복하기로 했다.
for i in range(1000):
    # 열린 페이지에서 HTML content를 html에 저장한다.
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    # Find all review elements on the page
    reviews = soup.select('div.review-list')

    # Loop through the review elements and extract the relevant data by beautifulsoup4
    for review in reviews:
        try :
            profile_body_information = review.select_one('p.review-profile__body_information').get_text()
        except :
            profile_body_information = 'empty'
        try :
            product_name = review.select_one('a.review-goods-information__name').text
        except :
            product_name = 'error'
        try :
            brand_name = review.select_one('a.review-goods-information__brand').text
        except :
            brand_name = 'error'
        try :
            review_text = review.select_one('div.review-contents__text').text
        except :
            review_text = 'empty'
        try :
            review_rating = review.select('span.review-list__rating__active')
            attr = review_rating
            for i in attr:
                if i['style']:
                    review_rating_ = i['style']
        except :
            review_rating_ = 'error'
        try :
            img_url = review.select('img.review-goods-information__image')
            attr = img_url
            for i in attr:
                if i['src']:
                    img_Url.append('https:' + i['src'])
        except :
            pass
        
        #크롤링한 데이터를 dataframe에 저장한다.
        review_df.iloc[n,0] = profile_body_information
        review_df.iloc[n,1] = product_name
        review_df.iloc[n,2] = brand_name
        review_df.iloc[n,3] = review_text
        review_df.iloc[n,4] = review_rating_

        #크롤링한 데이터를 count해준다.
        n += 1
    # '다음 10개 페이지' 버튼을 클릭하여 페이지를 이동한다.
    # 1, 11, 21, 31 ... 순으로 페이지를 탐색하게 된다.
    next_btn = driver.find_element(By.XPATH, '//div[@class="pagination"]//a[@class="fa fa-angle-right paging-btn btn next "]')
    next_btn.send_keys(Keys.ENTER)
    # 진척도를 알기 위해 20개마다 크롤링한 데이터 개수를 출력한다.
    print(n, '개')

#완료되면 웹 페이지를 닫는다.
driver.close()
print('end :', n) # 데이터 크롤링 결과를 출력한다.

#크롤링한 데이터들을 엑셀파일로 저장한다.
review_df.to_excel("./musinsa_results.xlsx")

image_url = pd.DataFrame(img_Url, columns=['url'])
image_url.to_excel('./musinsa_image_url.xlsx')


Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US) AppleWebKit/532.0 (KHTML, like Gecko) Chrome/3.0.195.21 Safari/532.0


C:\Users\user\AppData\Local\Temp\ipykernel_23276\379570517.py:7: DeprecationWarning: use options instead of chrome_options
  driver = wd.Chrome(chrome_options=options)


20 개
40 개
60 개
80 개
100 개
120 개
140 개
160 개
180 개
200 개
220 개
240 개
260 개
280 개
300 개
320 개
340 개
360 개
380 개
400 개
420 개
440 개
460 개
480 개
500 개
520 개
540 개
560 개
580 개
600 개
620 개
640 개
660 개
680 개
700 개
720 개
740 개
760 개
780 개
800 개
820 개
840 개
860 개
880 개
900 개
920 개
940 개
960 개
980 개
1000 개
1020 개
1040 개
1060 개
1080 개
1100 개
1120 개
1140 개
1160 개
1180 개
1200 개
1220 개
1240 개
1260 개
1280 개
1300 개
1320 개
1340 개
1360 개
1380 개
1400 개
1420 개
1440 개
1460 개
1480 개
1500 개
1520 개
1540 개
1560 개
1580 개
1600 개
1620 개
1640 개
1660 개
1680 개
1700 개
1720 개
1740 개
1760 개
1780 개
1800 개
1820 개
1840 개
1860 개
1880 개
1900 개
1920 개
1940 개
1960 개
1980 개
2000 개
2020 개
2040 개
2060 개
2080 개
2100 개
2120 개
2140 개
2160 개
2180 개
2200 개
2220 개
2240 개
2260 개
2280 개
2300 개
2320 개
2340 개
2360 개
2380 개
2400 개
2420 개
2440 개
2460 개
2480 개
2500 개
2520 개
2540 개
2560 개
2580 개
2600 개
2620 개
2640 개
2660 개
2680 개
2700 개
2720 개
2740 개
2760 개
2780 개
2800 개
2820 개
2840 개
2860 개
2880 개
2900 개
2920 개
2940 개
2960 개
2980 개
3000 개
302